In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()

clientmongo = os.getenv("CLIENTMONGO")
database= os.getenv("DATABASE")
collectiondb = os.getenv("COLLECTIONDB")

client = MongoClient(clientmongo)
db = client[database]
collection = db[collectiondb]

rows = []

# Traitement en lot
batch_size = 1000
cursor = collection.find({}, no_cursor_timeout=True).batch_size(batch_size)

for doc in cursor:
    content = doc.get("content", {})
    rows.append({
        "_id": doc.get("_id"),
        "titre": content.get("title"),
        "titre_sujet": content.get("courseware_title"),
        "Nom_du_cours": content.get("course_id"),
        "contenu_message": content.get("body"),
        "username": content.get("username"),
        "created_at": content.get("created_at"),
    })


df = pd.DataFrame(rows)
df.head()

/home/steve/Documents/FormationIA/ProjetMooc/Projet-MOOC/ENV/lib/python3.10/site-packages/pymongo/synchronous/collection.py:1945: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


,_id,titre,titre_sujet,Nom_du_cours,contenu_message,username,created_at
0,52ef4b71ab137b00720007d4,Jeune Ingénieur,Entre nous / Presentez-vous,CNAM/01002/Trimestre_1_2014,Dans le cadre de mes études d'ingénieur j'ai é...,qb,2014-02-03T07:55:29Z
1,52ef4d79b4907d2e23000996,Appronfondir mes connaissances,Entre nous / Presentez-vous,CNAM/01002/Trimestre_1_2014,"Bonjour,\nje suis actuellement une formation d...",fidji,2014-02-03T08:04:09Z
2,52ef4f99344caaf903000158,Motivations d'une archiviste enseignante,Entre nous / Presentez-vous,CNAM/01002/Trimestre_1_2014,Bonjour à tous (et je crois que nous sommes no...,ambruleaux,2014-02-03T08:13:13Z
3,52ef50b5cfc81d7e4100090e,Ancien Manager et Patron,Entre nous / Presentez-vous,CNAM/01002/Trimestre_1_2014,"Bonjour, j'ai eu une expérience de Manager de ...",EricBouchet,2014-02-03T08:17:57Z
4,52ef565b4b4451380f0008b2,Apprendre,Semaine_1 / Definition,CNAM/01002/Trimestre_1_2014,Je retiens que le manager peut jouer plusieurs...,edwigedk,2014-02-03T08:42:03Z


In [3]:
df["titre"].value_counts()

# Observation des titres les plus présents dans la base. Rien de concluant... 
# On apprend très peu sur ceux ci. 

titre
Présentation                                                    3774
Peter                                                            709
Introduction                                                     689
Bonjour                                                          671
Anne                                                             626
                                                                ... 
Hello from Georgia                                                 1
My Final grade isn´t updated with the Progress of the course       1
Vitesse de coupure - Big Bang                                      1
Home Makeover                                                      1
sens du vecteur champ électrique                                   1
Name: count, Length: 54314, dtype: int64

In [4]:
df["titre_sujet"].value_counts()
# On apprend beaucoup plus sur les thématiques/les cours

titre_sujet
Week 1 / Are you involved in a project                                                                4595
A vous de vous présenter / Présentations                                                              3176
Week 2 / The initiation process                                                                       2430
Discussion / Week 1: Favorite Applications                                                            1758
Bienvenue / Présentez-vous                                                                            1741
                                                                                                      ... 
Semaine 5 : Discussion sur les environnements de développement / Topic-Level Student-Visible Label       1
Semaine 6 : Questions sur Inkscape / Topic-Level Student-Visible Label                                   1
Semaine 4 / Semaine 4 : Conclusion                                                                       1
Semaine 4 / Semaine 4 : A

In [5]:
df["titre_sujet"].isna().sum()
# Présence de nombreuses valeurs nulles

np.int64(12241)

In [6]:
df = df.dropna(subset=['titre_sujet']).reset_index(drop=True)
#Suppression des valeurs nulles car cela entraîne des biais.


In [7]:
#Embeddings des titres du sujet pour traitement annexe

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings_titresujet = model.encode(df['titre_sujet'].fillna("Jonathan"), show_progress_bar=True)

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
#Enregistrement des embeddings pour ne par relancer le processus à chaque fois. 
np.save("../data/embeddings_titres.npy", embeddings_titresujet)

In [ ]:
# Chargement des embeddings
embeddings_titresujet = np.load("../data/embeddings_titres.npy")

In [ ]:
# Kmeans sur 2 clusters de façon à déterminer d'une part les questions, d'autre part
# les présentations. 
from sklearn.cluster import KMeans
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings_titresujet)
df["cluster"].value_counts()

In [ ]:
df.head(5)

In [ ]:
#Remplacement des clusters déterminés par des noms cohérents. 
df["cluster"] = df["cluster"].replace({1: "Présentation", 0: "Questions"})

In [ ]:
# Création d'un nouveau dataframe pour analyse complémentaire avec un reset index. 
df_questions = df[df["cluster"] == "Questions"]
df_questions = df_questions.reset_index(drop=True)

In [ ]:
df_questions.head()

In [ ]:
#Embedding des contenus des messages et enregistrement de ceux ci. 
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings_message = model.encode(df_questions['contenu_message'].fillna(""), show_progress_bar=True)

In [ ]:
np.save("../data/embeddings_messages.npy", embeddings_message)

In [ ]:
# Chargement des embeddings des messages
embeddings_message = np.load("../data/embeddings_messages.npy")

In [ ]:
# Topic modelling sur les contenus de messages (sinon trop long) et clustering

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Modèle multilingue
topic_model = BERTopic(embedding_model=model, verbose=True)

# Envelopper directement la liste de documents avec tqdm
documents = list(tqdm(df_questions["contenu_message"], desc="Préparation des documents"))
topics, probs = topic_model.fit_transform(documents, embeddings_message)
topic_model.save("../data/bertopic_original_model")

df_questions["topic"] = topics

# Visualisation du graphique des clusters
topic_model.visualize_topics().show()

In [ ]:
#from sklearn.cluster import KMeans
#from sklearn.preprocessing import normalize
#X = normalize(embeddings_message)
#kmeans = KMeans(n_clusters=60, random_state=42)
#df_questions["cluster_kmeans"] = kmeans.fit_predict(X)

In [ ]:
#Si vous souhaitez réduire le nombre de topics, vous le pouvez mais je le déconseille 
#fortement car on perd en pertinence

#topic_model.reduce_topics(documents, nr_topics=333)
#topic_model.visualize_topics().show()


In [ ]:
topic_model.get_topic_info()

In [ ]:
#Insertion des informations souhaitées dans le dataframe. 
topic_info = topic_model.get_topic_info()

def nommer_topic(topic_model, topic_id):
    mots = topic_model.get_topic(topic_id)
    if not mots:
        return "Autres"
    return ", ".join([m[0] for m in mots[:4]])

topic_noms = {tid: nommer_topic(topic_model, tid) for tid in topic_info["Topic"]}
df_questions["nom_topic"] = df_questions["topic"].map(topic_noms)

In [ ]:
df_questions.head(20)

In [ ]:
#enregistrement du dataframe questions pour l'utiliser dans l'application
df_questions.to_csv("../data/df_questions.csv", index=False)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

user = os.getenv("USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
port = os.getenv("PORT")
database = os.getenv("DATABASEBDD")
schema = os.getenv("SCHEMA")
table_name = os.getenv("TABLENAME")

# Connexion SQLAlchemy à PostgreSQL et création de la table si non existante
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

with engine.connect() as connection:
    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema};"))

#Enregistrement du DataFrame dans la table (dans le schéma) 

df_questions.to_sql(table_name, engine, schema=schema, if_exists="replace", index=False)

print(f"Données insérées dans {schema}.{table_name}")


In [ ]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String
from pgvector.sqlalchemy import Vector

Base = declarative_base()
with engine.connect() as connection:
    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema};"))

class Sujet(Base):
    __tablename__ = 'embeddings_titres'
    __table_args__ = {'schema': 'mooc_forum'}

    id = Column(Integer, primary_key=True, autoincrement=True)
    source_id = Column(String)  
    titre = Column(String)
    embedding = Column(Vector(384))  
Base.metadata.create_all(engine)

In [ ]:
import numpy as np
from sqlalchemy import text

embeddings = np.load("embeddings_titres.npy")
query_vector = embeddings[0].tolist()
sql_vector = f"[{', '.join(str(x) for x in query_vector)}]"

with engine.connect() as conn:
    results = conn.execute(text(f"""
        SELECT source_id, titre
        FROM mooc_forum.embeddings_titres
        ORDER BY embedding <=> '{sql_vector}'::vector
        LIMIT 5
    """)).fetchall()

for r in results:
    print(r.titre)
